# Setup

In [1]:
# Install all necessary packages
using Pkg
Pkg.activate(".")
Pkg.instantiate()
using Revise, ArmMotionStabilityRecoveryPerturbations, Biomechanics, ProgressMeter

Activating environment at `~/.julia/dev/ArmMotionStabilityRecoveryPerturbations/Project.toml`


In [2]:
using Distributed
# Add workers for parallel processing
prs = addprocs(;topology=:master_worker, exeflags=["-O3", "--project=@."])

# Load relevant code on all processes
@everywhere using Revise, ArmMotionStabilityRecoveryPerturbations

In [4]:
diskLoc = "/media/allen/Seagate Backup Plus Drive/"

rootdir = joinpath(diskLoc,"projects", "Arm-role-stability")
# rootdir = "location of data"

# Read all perturbations
perturbations = readperturbations(rootdir)

# Setup progressmeter and lock
pdesc = "Processing data... ";

In [6]:
[ (Subject=r[1], numperts=length(r[2]))
    for r in [ (sub, findall(x -> x.trial.subject == sub, perturbations)) for sub in 1:15 ] ]

15-element Array{NamedTuple{(:Subject, :numperts),Tuple{Int64,Int64}},1}:
 (Subject = 1, numperts = 20) 
 (Subject = 2, numperts = 60) 
 (Subject = 3, numperts = 59) 
 (Subject = 4, numperts = 60) 
 (Subject = 5, numperts = 60) 
 (Subject = 6, numperts = 60) 
 (Subject = 7, numperts = 44) 
 (Subject = 8, numperts = 60) 
 (Subject = 9, numperts = 60) 
 (Subject = 10, numperts = 60)
 (Subject = 11, numperts = 60)
 (Subject = 12, numperts = 60)
 (Subject = 13, numperts = 60)
 (Subject = 14, numperts = 60)
 (Subject = 15, numperts = 60)

## Analysis

In [31]:
p = Progress(length(perturbations)+1; desc=pdesc, barglyphs=BarGlyphs("[=>.]"))
uplock = ReentrantLock()

# Update the progressmeter in a thread-safe manner
@everywhere function updateprogress()
    lock(uplock)
    next!(p)
    unlock(uplock)
    nothing
end

# This is used by the workers
@everywhere function analyzeandupdate(p)
    numstrides = 5
    ap = analyzetrial(p, numstrides)
    
    # Tell the master process to update the progressmeter
    remotecall_wait(updateprogress,1)
    return ap
end

In [33]:
# Fit all the perturbations, don't quit on an error, add it to the results
next!(p)
analyzedperts = pmap(analyzeandupdate, perturbations; on_error=identity)
finish!(p)

Processing data... 100%[================================] Time: 0:02:01


In [34]:
# Check to see if any perturbations failed (to go back and see why they failed)
badtrials = findall(x -> !isa(x, AnalyzedSegment), analyzedperts)
if !isempty(badtrials)
    @show badtrials
    @show analyzedperts[badtrials]
end

In [35]:
# We don't need the other workers anymore
rmprocs(prs)

Task (done) @0x00007f6164788c40

## Results printing setup and print to file

In [36]:
using Dates, Statistics, DelimitedFiles

In [37]:
"""
    padarray(x, dims, pad)

Return a copy of `x` padded to a size of `dim` where padded elements are set to `pad`.
"""
function padarray(x::AbstractArray{T, N}, dsize::NTuple{DN, Int}, pad::T) where {T, N, DN}
    @assert DN >= N
    @assert prod(dsize .>= size(x))

    out = similar(x, dsize)
    out .= pad

    out[axes(x)...] .= x

    return out
end

padarray

In [38]:
# Setup loop variables
scalarstepvars = [
    :pre_steptimes,
    :pre_stepwidth,
    :pre_stepfoot,
    :post_steptimes,
    :post_stepwidth,
    :post_stepfoot
    ]

scalarstridevars = [
    :pre_stridetimes,
    :pre_stancetime,
    :pre_swingtime,
    :post_stridetimes,
    :post_stancetime,
    :post_swingtime
]

vectorvars = [
    :pre_lvmean,
    :pre_avmean,
    :pre_lvstd,
    :pre_avstd,
    :pre_lvrms,
    :pre_avrms,
    :pre_lvmax,
    :pre_avmax,
    :pre_lvmin,
    :pre_avmin,
    :pre_angmom_mean,
    :pre_angmom_std,
    :pre_angmom_rms,
    :pre_angmom_max,
    :pre_angmom_min,
    :post_lvmean,
    :post_avmean,
    :post_lvstd,
    :post_avstd,
    :post_lvrms,
    :post_avrms,
    :post_lvmax,
    :post_avmax,
    :post_lvmin,
    :post_avmin,
    :post_angmom_mean,
    :post_angmom_std,
    :post_angmom_rms,
    :post_angmom_max,
    :post_angmom_min,
    :pre_com_mean,
    :pre_com_std,
    :pre_com_rms,
    :pre_com_max,
    :pre_com_min,
    :post_com_mean,
    :post_com_std,
    :post_com_rms,
    :post_com_max,
    :post_com_min
    ]

armconds = [ :norm, :tied, :noswing ]
symconds = [ :sym, :asym ]
pertconds = [ :trip, :slip ]

ordinals = [
    "first",
    "second",
    "third",
    "fourth",
    "fifth"
    ]

subs = 1:15
numsubs = length(subs)
numstrides = 5
header = 6

6

In [39]:
whichpert = 5

5

In [40]:
# Initialize the results string
results = Vector{String}(undef, 1)
results[1] = "Analysis of the $(ordinals[whichpert]) stride after a perturbation\nGenerated: $(now())\n"

"Analysis of the fifth stride after a perturbation\nGenerated: 2019-10-16T14:42:37.1\n"

In [41]:
io = IOBuffer()
for vari in eachindex(scalarstepvars)
    subresults = fill(",", 5)
    R = collect(1:15)

    for arms in eachindex(armconds), symmetry in eachindex(symconds), ptype in eachindex(pertconds)
                # Grab all the perturbations for this combination of conditions
        relevant = findall(analyzedperts) do ap
            ap.s.trial.conds[:arms] == armconds[arms] &&
            ap.s.trial.conds[:sym] == symconds[symmetry] &&
            ap.s.trial.conds[:ptype] == pertconds[ptype]
        end
        
        wid = maximum( length(ap.results[scalarstepvars[vari]]) for ap in analyzedperts[relevant])
        et = Union{Missing, eltype(analyzedperts[relevant[1]].results[scalarstepvars[vari]])}
        
        _r = Array{et}(missing, numsubs, wid)
        
        # Only print the variable/condition if it is the first of its type; this will allow a
        # merged cell to be created to encompass all the columns below with that condition
        subresults[1] *= prod([ arms, symmetry, ptype] .== ones(Int,3)) ? string(scalarstepvars[vari])*"," : ","
        subresults[2] *= prod([ symmetry, ptype] .== ones(Int,2)) ? string(armconds[arms], ","^wid) : ","^wid
        subresults[3] *= (ptype == one(Int)) ? string(symconds[symmetry], ","^wid) : ","^wid
        subresults[4] *= string(pertconds[ptype], (","^wid))
        subresults[5] *= (string("Step #", i, ',') for i in 1:wid) |> prod

        for sub in subs
            try
                sr = sort(filter(ap -> ap.s.trial.subject == sub, analyzedperts[relevant]);
                            by=(x -> x.s.conds[:specificpnumber]))[whichpert]
                tmp = padarray(Vector{et}(sr.results[scalarstepvars[vari]]), (wid,), missing)
                
                _r[sub,:] .= tmp
            catch e
            end
        end
        
        empt = et.b === Symbol ? Symbol() : et.b(NaN)
        _r[ismissing.(_r)] .= empt

        R = [R _r]
    end
    
    writedlm(io, R, ',')
    
    results = [results; "\n\n"; subresults; String(take!(io)) ]
end

In [42]:
io = IOBuffer()
for vari in eachindex(scalarstridevars)
    subresults = fill(",", 5)
    R = collect(1:15)

    for arms in eachindex(armconds), symmetry in eachindex(symconds), ptype in eachindex(pertconds)
                # Grab all the perturbations for this combination of conditions
        relevant = findall(analyzedperts) do ap
            ap.s.trial.conds[:arms] == armconds[arms] &&
            ap.s.trial.conds[:sym] == symconds[symmetry] &&
            ap.s.trial.conds[:ptype] == pertconds[ptype]
        end
        
        wid = maximum( length(ap.results[scalarstridevars[vari]]) for ap in analyzedperts[relevant])
        et = Union{Missing, eltype(analyzedperts[relevant[1]].results[scalarstridevars[vari]])}
        
        _r = Array{et}(missing, numsubs, wid)
        
        # Only print the variable/condition if it is the first of its type; this will allow a
        # merged cell to be created to encompass all the columns below with that condition
        subresults[1] *= prod([ arms, symmetry, ptype] .== ones(Int,3)) ? string(scalarstridevars[vari])*"," : ","
        subresults[2] *= prod([ symmetry, ptype] .== ones(Int,2)) ? string(armconds[arms], ","^wid) : ","^wid
        subresults[3] *= (ptype == one(Int)) ? string(symconds[symmetry], ","^wid) : ","^wid
        subresults[4] *= string(pertconds[ptype], (","^wid))
        subresults[5] *= (string("Stride #", i, ',') for i in 1:wid) |> prod

        for sub in subs
            try
                sr = sort(filter(ap -> ap.s.trial.subject == sub, analyzedperts[relevant]);
                            by=(x -> x.s.conds[:specificpnumber]))[whichpert]
                tmp = padarray(Vector{et}(sr.results[scalarstridevars[vari]]), (wid,), missing)
                
                _r[sub,:] .= tmp
            catch
            end
        end
        
        _r[ismissing.(_r)] .= NaN

        R = [R _r]
    end
    
    writedlm(io, R, ',')
    
    results = [results; "\n\n"; subresults; String(take!(io)) ]
end

In [43]:
io = IOBuffer()
for vari in eachindex(vectorvars)
    subresults = fill(",", header)
    R = collect(1:15)

    for arms in eachindex(armconds), symmetry in eachindex(symconds), ptype in eachindex(pertconds)
        _r = Array{Union{Missing,Float64}}(missing, numsubs, 3*numstrides)
        # Only print the variable/condition if it is the first of its type; this will allow a
        # merged cell to be created to encompass all the columns below with that condition
        subresults[1] *= prod([ arms, symmetry, ptype] .== ones(Int,3)) ? string(vectorvars[vari])*"," : ","
        subresults[2] *= prod([ symmetry, ptype] .== ones(Int,2)) ? string(armconds[arms], ","^(3*numstrides)) : ","^(3*numstrides)
        subresults[3] *= (ptype == one(Int)) ? string(symconds[symmetry], ","^(3*numstrides)) : ","^(3*numstrides)
        subresults[4] *= string(pertconds[ptype], (","^(3*numstrides)))
        subresults[5] *= (string("Stride #", i, ','^3) for i in 1:numstrides) |> prod
        subresults[6] *= "X,Y,Z,"^numstrides

        # Grab all the perturbations for this combination of conditions
        relevant = findall(analyzedperts) do ap
            ap.s.trial.conds[:arms] == armconds[arms] &&
            ap.s.trial.conds[:sym] == symconds[symmetry] &&
            ap.s.trial.conds[:ptype] == pertconds[ptype]
        end

        for sub in subs
            try
                sr = sort(filter(ap -> ap.s.trial.subject == sub, analyzedperts[relevant]);
                        by=(x -> x.s.conds[:specificpnumber]))[whichpert]
                tmp = Vector{Union{Missing,Float64}}(vcat(sr.results[vectorvars[vari]]...))

                _r[sub,:] .= tmp
            catch
            end
        end
        
        _r[ismissing.(_r)] .= NaN

        R = [R _r]
    end
    
    writedlm(io, R, ',')
    
    results = [results; "\n\n"; subresults; String(take!(io)) ]
end

In [44]:
(path, io) = mktemp()

for line in results
    println(io, line)
end

close(io)

resfn = joinpath(pwd(), "perturbation-analysis_pert_$whichpert.csv")

mv(path, resfn; force=true)

"/home/allen/.julia/dev/ArmMotionStabilityRecoveryPerturbations/perturbation-analysis_pert_5.csv"